## Code for developing a macro for imageJ 

In [1]:
from FIJI_Marco_For_Particle_Identification import process_NF_particles as pnp
import glob
import multiprocessing as mp
import numpy as np 
import os
import pandas as pd
import scipy as sc
import scipy.io as sio
import shutil
import time 
import scyjava

In [2]:
# run this only for beta testing 
import imagej
#ij = imagej.init('net.imagej:imagej:2.3.0', mode='interactive')

In [3]:
ij = imagej.init('C:/Installer_file/FIJI/Fiji.app', mode = 'interactive')

## Execution of the legacy code for testing purposes while making the amendments 


In [4]:
# This is he most important part of the code. parallelize this part
# LEGACY CODE
# create a function to input the folder path and the image set
def process_NF_particles(path, frame_start, frame_finish, mem_alloc):
    
    
    scyjava.config.add_options(mem_alloc);
    
    A_pre = '\nsetBatchMode(false);\nopen("';
    #Z = np.array([]);
    df = pd.DataFrame();
     
    # count the total number fo frames  
    N = frame_finish - frame_start;
    
    # run the timer
    start_time = time.time();

    # define a part of the macros 
    macro_main = '''.tif");
    // Invert the image 
    run("Invert");
    // Enhance the contrast of the image
    run("Enhance Contrast", "saturated=0.35");
    // Apply the gaussian blur: low pass filter: sigma set at 2 
    //run("Accurate Gaussian Blur...", "sigma=2");
    run("Accurate Gaussian Blur", "sigma=2");
    // find the peaks after applying the filter 
    run("Find Maxima...", "prominence=10 output=List");
    // Save the data as a csv file 
    '''
    
    # define the saving route 
    basename = '''saveAs("Results", "''';
    ending = '''");\n''';
    filename = "C:/Temporary/Results_"+ str(1)+ "_to_"+ str(2)+".tsv";
    full_name = basename+filename+ending;
    
    # name the location of the memory mapped array.
    filename_memmap = "C:/Temporary/Array_"+ str(1)+ "_to_"+ str(2)+".array";
    
    # create a memory mapped array
    Z = np.memmap(filename_memmap, dtype='float32', mode='w+', shape = (1,4));
    
    # loop for a number of frames
    for i in range(frame_start, frame_finish):

        # build a image string 
        n = str(i).zfill(6);

        # find the positions of the particle using the macro
        result = ij.py.run_macro(A_pre+path+n+macro_main+full_name);

        # read the text file into a pandas dataframe
        df = pd.read_csv(filename, dtype = np.float32, sep='\t');
        
        # add  the frame number to the array # df contains the data of one frame so it is very tiny
        df['F'] = i*np.ones(len(df.index), dtype = np.float32);
        
        # find the length of the master array
        master_len = np.shape(Z)[0];
        frame_len = np.shape(df)[0];
        
        # concatenate with the master array
        #Z = pd.concat([Z,df[['X', 'Y', 'F']]]);
        
        # Develop a code for the memory mapped approach
        # strategy: create a memory mapped array and keep updating it 
        Z = np.memmap(filename_memmap, dtype='float32', mode='r+', shape = ((master_len + frame_len),4));
        
        # dump the df into z
        Z[master_len:,:] = np.array(df);
        
        print ("chunk",frame_start,"to", (frame_finish-1),":", round((i-frame_start)/N*100)," percent complete  \r"); 
    
    # drop the first empty row.
    Z = Z[1:,:];
    
    #macro_end = '''
    #close("Results");
    #close();
    #'''
    #result = ij.py.run_macro(macro_end);             
    end_time = time.time();
    print("the time lapse is",(end_time - start_time)/N, "per frame");
    

In [5]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ test execution ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
Path = "D:/binary_flock_density_manipulation/density_0.40_date_10.02.2022/15h21_2x2x_2exp_160V_350fps_files/Images/";

memory_allocation = "-Xmx" + str(1) + "g";
A  = process_NF_particles(Path, 10,20,memory_allocation)


chunk 10 to 19 : 0  percent complete  
chunk 10 to 19 : 10  percent complete  
chunk 10 to 19 : 20  percent complete  
chunk 10 to 19 : 30  percent complete  
chunk 10 to 19 : 40  percent complete  
chunk 10 to 19 : 50  percent complete  
chunk 10 to 19 : 60  percent complete  
chunk 10 to 19 : 70  percent complete  
chunk 10 to 19 : 80  percent complete  
chunk 10 to 19 : 90  percent complete  
the time lapse is 0.38921418190002444 per frame


In [ ]:
Aa[1]

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ test saving ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Transform the path name to the position for saving the .mat output
parts = os.path.normpath(Path).split(os.sep)[:-1];
parts.extend(['Analysis','1_Positions','Positions_NF_test.mat']);
mat_file_name = '/'.join(parts);

# Save the data in a .mat file 
dictn = {"XYZ":np.array(master_array)};
sio.savemat(mat_file_name, dictn);

In [ ]:
def process_NF_particles(path, frame_start, frame_finish):
    
    A_pre = '\nsetBatchMode(true);\nopen("';
    Z = pd.DataFrame();
    df = pd.DataFrame();
     
    # count the total number fo frames  
    N = frame_finish - frame_start;
    
    # run the timer
    start_time = time.time();

    # define a part of the macros 
    macro_main = '''.tif");
    // Invert the image 
    run("Invert");
    // Enhance the contrast of the image
    run("Enhance Contrast", "saturated=0.35");
    // Apply the gaussian blur: low pass filter: sigma set at 2 
    run("Gaussian Blur...", "sigma=2");
    // find the peaks after applying the filter 
    run("Find Maxima...", "prominence=10 output=List");
    // Save the data as a csv file 
    '''
    
    # define the saving route 
    basename = '''saveAs("Results", "''';
    ending = '''");\n''';
    filename = "C:/Temporary/Results_"+ str(1)+ "_to_"+ str(2)+".tsv";
    full_name = basename+filename+ending;
    
    # loop for a number of frames
    for i in range(frame_start, frame_finish):

        # build a image string 
        n = str(i).zfill(6);

        # find the positions of the particle using the macro
        result = ij.py.run_macro(A_pre+path+n+macro_main+full_name);

        # read the text file into a pandas dataframe
        df = pd.read_csv(filename, sep='\t');
        
        # add  the frame number to the array
        df['F'] = i*np.ones(len(df.index));
        
        # concatenate with the master array
        Z = pd.concat([Z,df[['X', 'Y', 'F']]]);

        # close all the frames that you have opened 
        if (i%100 == 0):
            print ("chunk",frame_start,"to", (frame_finish-1),":", round((i-frame_start)/N*100)," percent complete  \r"); 
            macro_2 = '''
              macro "Close All Windows" { 
                  while (nImages>0) { 
                      selectImage(nImages); 
                      close(); 
                  } 
              } 
            '''
            result = ij.py.run_macro(macro_2);

    macro_end = '''
    macro "Close All Windows" { 
      while (nImages>0) { 
         selectImage(nImages); 
          close();  
      } 
      close("Results");
    }
    '''
    result = ij.py.run_macro(macro_end);             
    end_time = time.time();
    print("the time lapse is",(end_time - start_time)/N, "per frame");
    return Z